## 데이터 불러오기 (Read Data)

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')


from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import f1_score
import lightgbm as lgb
# from lightgbm import LGBMClassifier as lgb

# import optuna.integration.lightgbm as lgb

# from imblearn.over_sampling import SMOTE
SEED = 1996

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

## 데이터 구조 확인 (Checking the shapes of data)

In [3]:
all_df = pd.concat([train, test])
all_df.reset_index(inplace = True, drop = True)
del(all_df['id'])

In [25]:
all_df.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,gain-loss,work,human
0,40,Private,12.034922,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0.0,0.0,60,United-States,>50K,0.0,Private_Sales,White_Male
1,17,Private,11.529065,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0.0,0.0,20,United-States,<=50K,0.0,Private_Machine-op-inspct,White_Male
2,18,Private,12.775240,Some-college,10,Never-married,Other-service,Own-child,White,Male,0.0,0.0,16,United-States,<=50K,0.0,Private_Other-service,White_Male


In [4]:
all_df['gain-loss'] = all_df['capital_gain'] - all_df['capital_loss']
all_df['work'] = all_df['workclass'].apply(str) + '_' + all_df['occupation'].apply(str)
all_df['human'] = all_df['race'].apply(str) + '_' + all_df['sex'].apply(str)

all_df.loc[all_df['native_country'] == '?', 'native_country'] = np.nan
all_df.loc[all_df['workclass'] == '?', 'native_country'] = np.nan
all_df.loc[all_df['occupation'] == '?', 'native_country'] = np.nan

In [5]:
log_var = ['capital_gain','capital_loss','gain-loss','fnlwgt']
for col in log_var:
    all_df[col] = np.log1p(all_df[col])

In [26]:
del(all_df['education'])

In [27]:
train = all_df.iloc[:26049,:]
test = all_df.iloc[26049:,:]

## 분류 작업에 필용한 함수 불러오기 (Import methods for classification task)

In [28]:
from pycaret.classification import *

In [29]:
clf = setup(data = train, target = 'income', session_id = 25,silent = True)

Setup Succesfully Completed!


,Description,Value
0,session_id,25
1,Target Type,Binary
2,Label Encoded,"<=50K: 0, >50K: 1"
3,Original Data,"(26049, 17)"
4,Missing Values,True
5,Numeric Features,6
6,Categorical Features,10
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [30]:
best_3 = compare_models(sort = 'F1', n_select = 3)#87985 seed 25 고정한 기준 education제거

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.8747,0.9287,0.6660,0.7842,0.7200,0.6400,0.6437,0.2324
1,CatBoost Classifier,0.8744,0.9296,0.6506,0.7935,0.7148,0.6352,0.6405,9.6558
2,Extreme Gradient Boosting,0.8711,0.9265,0.6610,0.7739,0.7126,0.6302,0.6337,3.7434
3,Ada Boost Classifier,0.8617,0.9152,0.6234,0.7623,0.6857,0.5982,0.6033,2.4958
4,Gradient Boosting Classifier,0.8655,0.9208,0.5901,0.8024,0.6798,0.5972,0.6086,6.7563
5,Random Forest Classifier,0.8509,0.8835,0.5898,0.7424,0.6568,0.5633,0.5697,0.2343
6,Extra Trees Classifier,0.8389,0.8831,0.6218,0.6844,0.6512,0.5468,0.5482,1.8501
7,Linear Discriminant Analysis,0.8435,0.8969,0.5928,0.7131,0.6471,0.5477,0.5518,0.7434
8,Logistic Regression,0.8411,0.8963,0.5887,0.7065,0.6419,0.5409,0.5448,0.7382
9,Ridge Classifier,0.8426,0.0000,0.5409,0.7394,0.6245,0.5280,0.5384,0.1489


In [15]:
best_3 = compare_models(sort = 'F1', n_select = 3)#87985 seed 25 고정한 기준

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.8747,0.9287,0.6660,0.7842,0.7200,0.6400,0.6437,0.2593
1,CatBoost Classifier,0.8737,0.9295,0.6510,0.7906,0.7138,0.6338,0.6389,9.8161
2,Extreme Gradient Boosting,0.8711,0.9265,0.6610,0.7739,0.7126,0.6302,0.6337,4.2117
3,Ada Boost Classifier,0.8617,0.9152,0.6234,0.7623,0.6857,0.5982,0.6033,2.9611
4,Gradient Boosting Classifier,0.8655,0.9207,0.5898,0.8023,0.6797,0.5970,0.6085,7.4182
5,Random Forest Classifier,0.8500,0.8783,0.5898,0.7384,0.6555,0.5612,0.5672,0.2488
6,Linear Discriminant Analysis,0.8435,0.8969,0.5928,0.7131,0.6471,0.5477,0.5518,0.8673
7,Extra Trees Classifier,0.8357,0.8808,0.6216,0.6751,0.6468,0.5401,0.5411,2.0304
8,Logistic Regression,0.8421,0.8970,0.5883,0.7106,0.6433,0.5431,0.5473,0.7647
9,SVM - Linear Kernel,0.8215,0.0000,0.6358,0.6616,0.6262,0.5127,0.5281,1.6793


## 모델 앙상블 (Model Ensemble)

In [31]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')#education 제거

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8774,0.9278,0.6803,0.7843,0.7286,0.6500,0.6527
1,0.8730,0.9307,0.6478,0.7901,0.7119,0.6315,0.6366
2,0.8777,0.9326,0.6716,0.7917,0.7267,0.6486,0.6523
3,0.8752,0.9273,0.6591,0.7908,0.7190,0.6396,0.6440
4,0.8719,0.9264,0.6361,0.7935,0.7061,0.6255,0.6317
Mean,0.8751,0.9290,0.6590,0.7901,0.7185,0.6390,0.6435
SD,0.0023,0.0023,0.0159,0.0031,0.0086,0.0095,0.0083


In [16]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')#기본

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8772,0.9277,0.6803,0.7833,0.7282,0.6493,0.6520
1,0.8725,0.9308,0.6455,0.7895,0.7103,0.6296,0.6348
2,0.8777,0.9327,0.6727,0.7909,0.7271,0.6489,0.6524
3,0.8758,0.9273,0.6602,0.7921,0.7202,0.6412,0.6456
4,0.8708,0.9264,0.6327,0.7915,0.7032,0.6220,0.6283
Mean,0.8748,0.9290,0.6583,0.7895,0.7178,0.6382,0.6426
SD,0.0027,0.0024,0.0174,0.0032,0.0097,0.0108,0.0096


## 모델 예측 (Prediction)
- 구축된 앙상블 모델을 통해 예측을 해보겠습니다. 
- setup 환경에 이미 hold-out set이 존재하므로 해당 데이터에 대해 예측을 하여 모델 성능을 확인하겠습니다. 

----
- We will use the ensembled model on predicting unseen data.
- There is already a hold-out set constucted on our environment so we will test on it to evaluate the performance.

In [32]:
pred_holdout = predict_model(blended)# education제거 : 0.6993           기본 : 7002

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8663,0.9249,0.6422,0.7675,0.6993,0.6142,0.6182


## 전체 데이터에 대한 재학습 (Re-training the model on whole data)

- 현재까지 실험은 주어진 train 데이터를 다시 한 번 train / validation으로 나눠서 실험을 한 것이므로, 전체 train 데이터에 학습되어 있지 않습니다. 
- 최적의 성능을 위해 전체 데이터에 학습을 시켜주도록 하겠습니다. 

------
- Until now we have splitted the given train data into another train / validation sets to experiment. So the models are not trained on the full training data set.
- We will train the model on the whole dataset for the most optimal performance. 

In [33]:
final_model = finalize_model(blended)

## 대회용 test set에 대한 예측 (Predicting on test set for the competition)

- predict_model 함수를 통해 재학습된 모델을 대회용 test set에 대해 예측해보겠습니다. 
- We will now use the re-trained model on the test set for the competition

In [34]:
predictions = predict_model(final_model, data = test)

In [35]:
submission['voted'] = predictions['Score']

In [36]:
submission.loc[submission['voted'] >= 0.5, 'prediction'] = 1

In [37]:
del(submission['voted'])

In [38]:
submission

,id,prediction
0,0,0
1,1,1
2,2,0
3,3,1
4,4,0
...,...,...
6507,6507,1
6508,6508,1
6509,6509,0
6510,6510,0


In [39]:
submission.to_csv('base_87985_seed25_deledu.csv', index = False)